In [17]:
import pandas as pd

# Load the dataset
geo_device = pd.read_csv("input_data/estat_isoc_ci_dev_i_en.csv")
geo_beds = pd.read_csv("input_data/estat_tour_cap_nat_en.csv")

In [2]:
#check the data
print(geo_device.head())

                   DATAFLOW        LAST UPDATE freq   ind_type    indic_is  \
0  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   
1  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   
2  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   
3  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   
4  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   

     unit geo  TIME_PERIOD  OBS_VALUE OBS_FLAG  CONF_STATUS  
0  PC_IND  AL         2018        NaN        u          NaN  
1  PC_IND  AT         2016      37.41      NaN          NaN  
2  PC_IND  AT         2018      42.45      NaN          NaN  
3  PC_IND  AT         2021      39.62      NaN          NaN  
4  PC_IND  AT         2023      40.34      NaN          NaN  


In [3]:
print(geo_beds.columns)
print(geo_device.columns)

Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'accomunit', 'unit', 'nace_r2',
       'geo', 'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG', 'CONF_STATUS'],
      dtype='object')
Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'ind_type', 'indic_is', 'unit',
       'geo', 'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG', 'CONF_STATUS'],
      dtype='object')


In [18]:
# cols to keep
geo_beds = geo_beds[['geo','TIME_PERIOD','accomunit', 'unit', 'nace_r2','OBS_VALUE']]
print(geo_beds.head())

  geo  TIME_PERIOD accomunit unit nace_r2  OBS_VALUE
0  AT         1990     BEDPL   NR    I551   650559.0
1  AT         1991     BEDPL   NR    I551   654127.0
2  AT         1992     BEDPL   NR    I551   653586.0
3  AT         1993     BEDPL   NR    I551   651003.0
4  AT         1994     BEDPL   NR    I551   650020.0


In [13]:
# print(geo_beds['geo'].value_counts())
print(geo_beds['nace_r2'].value_counts())
# print(geo_beds['accomunit'].value_counts())
# print(geo_beds['unit'].value_counts())
# print(geo_beds['TIME_PERIOD'].value_counts())

nace_r2
I551         6867
I552_I553    4320
I551-I553    4294
I553         4265
I552         4139
Name: count, dtype: int64


In [8]:
geo_beds.dtypes

geo            object
TIME_PERIOD     int64
accomunit      object
unit           object
nace_r2        object
dtype: object

In [ ]:
# Rows to use: You should only use rows where: 
# “accomunit” is “BEDPL”, --> accomodation unit : bedplaces
# “unit” is “NR”, --> number
# “nace_r2” is “I551”, --> accomodation type : hotels and similar accommodation 
# “TIME_PERDIO” is “2016” 

# Output the number of bedplaces in hotels and similar accommodation in 2016 for each country
geo_beds_2016 = geo_beds[(geo_beds['accomunit'] == 'BEDPL') & (geo_beds['unit'] == 'NR') & (geo_beds['nace_r2'] == 'I551') & (geo_beds['TIME_PERIOD'] == 2016)]
print(geo_beds_2016.head())

    geo  TIME_PERIOD accomunit unit nace_r2  OBS_VALUE
26   AT         2016     BEDPL   NR    I551   601945.0
60   BE         2016     BEDPL   NR    I551   129232.0
94   BG         2016     BEDPL   NR    I551   281869.0
126  CH         2016     BEDPL   NR    I551   271710.0
159  CY         2016     BEDPL   NR    I551    83251.0


In [5]:
#check the data
print(geo_device.head())

                   DATAFLOW        LAST UPDATE freq   ind_type    indic_is  \
0  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   
1  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   
2  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   
3  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   
4  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   

     unit geo  TIME_PERIOD  OBS_VALUE OBS_FLAG  CONF_STATUS  
0  PC_IND  AL         2018        NaN        u          NaN  
1  PC_IND  AT         2016      37.41      NaN          NaN  
2  PC_IND  AT         2018      42.45      NaN          NaN  
3  PC_IND  AT         2021      39.62      NaN          NaN  
4  PC_IND  AT         2023      40.34      NaN          NaN  


In [21]:
geo_device.columns

Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'ind_type', 'indic_is', 'unit',
       'geo', 'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG', 'CONF_STATUS'],
      dtype='object')

In [22]:
# cols to keep
geo_device = geo_device[['geo','ind_type','indic_is', 'unit', 'TIME_PERIOD','OBS_VALUE', 'OBS_FLAG']]

In [23]:
# rows to use: You should only use rows where:
# · “ind_type” is “IND_TOTAL”
# · “indic_is” is “I_IUG_TV”
# · “unit” is “PC_IND”
# · “TIME_PERIOD” is “2016”

# Output the number of TV sets per 100 inhabitants in 2016 for each country
geo_device_2016 = geo_device[(geo_device['ind_type'] == 'IND_TOTAL') & (geo_device['indic_is'] == 'I_IUG_TV') & (geo_device['unit'] == 'PC_IND') & (geo_device['TIME_PERIOD'] == 2016)]
print(geo_device_2016.head())


      geo   ind_type  indic_is    unit  TIME_PERIOD  OBS_VALUE OBS_FLAG
97734  AT  IND_TOTAL  I_IUG_TV  PC_IND         2016      14.78      NaN
97735  BE  IND_TOTAL  I_IUG_TV  PC_IND         2016       9.89      NaN
97736  BG  IND_TOTAL  I_IUG_TV  PC_IND         2016       3.71      NaN
97737  CY  IND_TOTAL  I_IUG_TV  PC_IND         2016       6.41      NaN
97738  CZ  IND_TOTAL  I_IUG_TV  PC_IND         2016       5.32      NaN


In [25]:
geo_device_2016['OBS_FLAG'].value_counts()

OBS_FLAG
b    2
Name: count, dtype: int64

In [27]:
import numpy as np

# Special values
# ● The value “: ” signifies missing data.
# ● Rows where OBS_FLAG contains “u” or “bu” are unreliable and should be considered as missing data.
# ● Country codes “EA”, "EU27_2007", "EU27_2020";, "EU28"; should be treated the same as missing data and should be ignored.[2]

#Treat the special values as missing data
geo_device_2016 = geo_device_2016[~geo_device_2016['OBS_FLAG'].fillna('').str.contains('u|bu')]
geo_device_2016 = geo_device_2016[~geo_device_2016['geo'].isin(['EA', 'EU27_2007', 'EU27_2020', 'EU28'])]

# Replace ":" with NaN in 'OBS_VALUE' and then drop rows with NaN
geo_device_2016['OBS_VALUE'] = geo_device_2016['OBS_VALUE'].replace(':', np.nan).astype(float)
geo_device_2016 = geo_device_2016.dropna(subset=['OBS_VALUE'])

In [28]:
# Doing the same for geo_beds_2016
geo_beds_2016 = geo_beds_2016[~geo_beds_2016['geo'].isin(['EA', 'EU27_2007', 'EU27_2020', 'EU28'])]
geo_beds_2016['OBS_VALUE'] = geo_beds_2016['OBS_VALUE'].replace(':', np.nan).astype(float)
geo_beds_2016 = geo_beds_2016.dropna(subset=['OBS_VALUE'])

In [30]:
#left merge the two dataframes on 'geo'
geo_merged = geo_beds_2016.merge(geo_device_2016, on='geo', suffixes=('_beds', '_device'))
# keep only the columns we need: “Country Code”, “Percentage of individuals” and “Number of Bed-places”.
geo_merged = geo_merged[['geo', 'OBS_VALUE_beds', 'OBS_VALUE_device']]
# Rename the columns to “Country Code”, “Number of Bed-places” and “Percentage of individuals”
geo_merged.columns = ['Country Code', 'Number of Bed-places', 'Percentage of individuals']
import os

# Create the output directory if it doesn't exist
if not os.path.exists('output_data'):
	os.makedirs('output_data')

# Save the merged dataframe to a CSV file
geo_merged.to_csv('output_data/geo_beds_device.csv', index=False)
print(geo_merged.head())

  Country Code  Number of Bed-places  Percentage of individuals
0           AT              601945.0                      14.78
1           BE              129232.0                       9.89
2           BG              281869.0                       3.71
3           CY               83251.0                       6.41
4           CZ              317756.0                       5.32


<!-- # Visualize the data for Outcome
# Can I Multiply "Number of Bed-places" by "% of Individuals" to Estimate Target Users?
# I can. BUT! This is a rough estimate and has some assumptions and limitations.

# What the Percentage Means:
- The "% of individuals" is the share of the general population who use a TV to access the internet, not specifically hotel guests. This assumes hotel guests are similar to the general population in their tech habits.
- Bed-places ≠ Unique Guests: "Bed-places" is the number of beds available, not the number of guests per year. Actual guest numbers could be higher (if beds are used by multiple guests per year) or lower (if occupancy is low).
- Not All Guests Will Use the Feature: Even if someone uses a smart TV at home, they may not use it in a hotel.
- Market Size Estimate: This calculation gives you a relative market size for comparison between countries. -->